# Simplify Your Tests With Fixtures

# https://github.com/obestwalter/pytest-fixtures-introduction

# https://oliver.bestwalter.de

# If you will not have liked this webinar ...

... because it was too short and not by Holger Krekel: [pytest tutorial](https://www.youtube.com/watch?v=AiThU6JQbE8) (3 hours)

... because it was too long and Paul did not talk enough: [pytest fixtures](https://www.jetbrains.com/pycharm/guide/tutorials/visual_pytest/fixtures/) (7 minutes)

... because it was too basic: [Advanced Uses of py.test (sic!) Fixtures](https://www.youtube.com/watch?v=IBC_dxr-4ps) by Floris Bruynooghe

# ~100 LOC parody implementation of pytest with fixtures ...

![](_img/metaprogramming-talk.png)

## https://github.com/obestwalter/abridged-meta-programming-classics

# Questions hopefully answered after this ...

* (15%) [theory] What problem do fixtures solve? Why should I use fixtures?

* (70%) [applied] What are the essentials I need to know about fixtures?

* (7.5%) [theory] How do I find and use built-in or 3rd party fixtures?

* (7.5%) [theory] How can I share my custom fixtures between projects?

# Why should I use fixtures?

> The History of every major Galactic Civilization tends to pass through three distinct and recognizable phases, those of Survival, Inquiry and Sophistication, otherwise known as the How, Why, and Where phases. For instance, the first phase is characterized by the question 'How can we eat?' the second by the question 'Why do we eat?' and the third by the question 'Where shall we have lunch?

-- Douglas Adams,  The Restaurant at the End of the Universe 

> Learning something new is always about Why, What and How. Why should I learn it? What can I do with it? How do I do it? Those three things are actually always somehow intertwined and everyone tends to prioritize them differently. Which is why there is no correct order in addressing them. So just go with it.

-- me, just now

# Arguments For Using Fixtures

# The Analogy Based / Traditional Argument For Using Fixtures

In the real world, craftspeople use a fixture to hold a workpiece in place to make working with them easier.

![](_img/welding-fixture.jpg)

In the software world, computer craftspeople use fixtures to pretty much [solve the same problem](https://docs.pytest.org/en/stable/fixture.html).

In [ ]:
def test_tmp_path_creates_a_directory(tmp_path):
    assert tmp_path.exists()

# henceforth thou shalt use fixtures

![](_img/bycycle-repairman.png)

#### [Monty Python - Bicycle Repairman](https://en.wikipedia.org/wiki/List_of_Monty_Python%27s_Flying_Circus_episodes#Bicycle_Repair_Man)

# The Logically Compelling Argument for Using Fixtures

Premise 1: test code is real code - not some special subcategory of code where different rules and quality standards apply

Premise 2: for comprehensibility and maintainability excessive code duplication must be avoided by creating useful abstractions ("new words" (Hettinger))

Premise 3: pytest fixtures provide a mechanism to create useful abstractions for automatic tests

# henceforth thou shalt use fixtures

![](_img/philosophers.jpg)

#### [Monty Python - Philosophers Football Match](https://en.wikipedia.org/wiki/The_Philosophers%27_Football_Match)

# The third of these two groups likes it more computer sciency 

# So we will have a quick look at ...

## frameworks vs libraries

## software design patterns in action

# pytest is a framework (not just a library)

![[Birgit Bachler 2010](https://www.birgitbachler.com/)](_img/framework-1.png)

#### [Birgit Bachler 2010](https://www.birgitbachler.com/portfolio/)

# pytest is a framework (not just a library)

![](_img/framework-2.png)

#### [Birgit Bachler 2010](https://www.birgitbachler.com/portfolio/)

# pytest is a framework (not just a library)

![](_img/framework-3.png)

#### [Birgit Bachler 2010](https://www.birgitbachler.com/portfolio/)

# Software Design Patterns in Action


In [3]:
def test_everything():  # <- pytest magically executes the test for us!
    assert everything() == "is gonna be ok!"

Any function named ``test_...`` will be called by pytest as part of a testrun. The decision about if and when that function is called is not made by us, but by pytest. 

The control flow is inverted compared to how we normally program.


This is a programming principle called ...

# [Inversion of Control](https://en.wikipedia.org/wiki/Inversion_of_control)

# Software Design Patterns in Action


In [4]:
def test_tmp_path_creates_a_directory(tmp_path):  # magic parameters!
    assert tmp_path.exists()

We could say that the test depends on an object provided by the ``tmp_path`` fixture and that it is requested by adding it to the parameters in the function definition.

pytest then injects that requested dependency into the test function as part of the testrun. 

This means in practical terms that pytest calls the test function with the correct argument (the return value of the ``tmp_path`` fixture function that pytest also calls for us beforehand). 

This is a programming technique or software design pattern called ...

# [Dependency Injection](https://en.wikipedia.org/wiki/Dependency_injection)

# quick digression ... what if we had to do it ourselves?


# A function expecting a tmp_path


In [ ]:
def some_function(tmp_path):
    print(f"I was provided with {tmp_path=}")
    assert tmp_path.exists()
    return tmp_path

# A tmp_path provider for arbitrary functions


In [ ]:
import tempfile
import shutil
from pathlib import Path

def tmp_path_provider(func):
    tmp_path = Path(tempfile.mkdtemp())
    result = func(tmp_path)
    shutil.rmtree(tmp_path)
    return result

tmp_path = tmp_path_provider(some_function)
tmp_path, tmp_path.exists()

# A better tmp_path provider for arbitrary functions


In [ ]:
import tempfile
import shutil
from pathlib import Path

def tmp_path_provider(func):
    with tempfile.TemporaryDirectory() as tmp_dirname:
        return func(Path(tmp_dirname))

tmp_path = tmp_path_provider(some_function)
tmp_path, tmp_path.exists()

# Turning the tmp_path provider into a fixture


In [ ]:
import tempfile
import shutil
from pathlib import Path

import pytest

@pytest.fixture
def tmp_path():
    with tempfile.TemporaryDirectory() as tmp_dirname:
        # magic! yield ensures correct setup/teardown
        yield Path(tmp_dirname)

# What do these things have in common?

* context managers

* decorators

* fixtures

## They are different ways of addressing the ...

## ... "do something before / do something after" problem

# The goal: make testing easier for everybody involved

# A quick real world example

## `tox/src/tox/_pytestplugin.py:initproj` fixture

# Enter PyCharm

# Fixture Essentials on One Slide

* two ways of requesting: parameter in function definition, ``@pytest.mark.usefixtures``

* two main ways of creation: ``@pytest.fixture``, ``@pytest.mark.parametrize``

* two main flavours: ``return``, ``yield``

* pytest fixture lookup works similar to Python name lookup (CMLIB)

* fixtures can depend on fixtures (scope gotcha, e.g. tmp_path in higher scopes)

* ``name``: specify what happens and what is returned


# Some Gotchas

* ``@pytest.mark.usefixture`` does [not work](https://github.com/pytest-dev/pytest/issues/3664) on a fixture function - you **have** to request the fixture via parameter

* importable test helpers should live in an importable package - do not import them via ``sys.path`` hacks and especially do not put them in ``conftest.py`` as this must never be imported directly

## Whatever Gotcha You Might Face ...

Use the fixture command line functions to see what happens and if possible run the tests through a debugger. Tests are normal code and they should be debuggable like normal code.

# How do I find 3rd party plugins?

* "officially" supported plugins: [pytest-dev](https://github.com/pytest-dev)

* ``pip search pytest``

* [pypi.org: pytest trove classifier](https://pypi.org/search/?q=&o=&c=Framework+%3A%3A+Pytest)

* [app making basic compatibility tests](https://plugincompat.herokuapp.com/)

# How can I share my custom fixtures between projects?

* transfer your fixtures (and hooks) into an installable package

* define [pytest11 ``entry_point``](https://github.com/pytest-dev/pytest-django/blob/ee7858af1a80e0091af4d260c0a70e5766c1196a/setup.py#L77) pointing to implementation module

## [pytest plugins tutorial](https://docs.pytest.org/en/stable/writing_plugins.html)

## [pytest cookiecutter](https://github.com/pytest-dev/cookiecutter-pytest-plugin)

# Hooks - If Fixtures are not Enough ...

## Hooks provide entry points to make it possible to deeply manipulate the behaviour of many apects of a testrun.

## Best way to go about this is ...

* check out the [hook docs](https://docs.pytest.org/en/stable/reference.html#hooks)

* monkey see, monkey do: look at existing plugins that do similar things and check out their uses of hooks

# Some Fixture Archeology

## Fixtures started off as funcargs

* [introduction of funcargs](https://docs.pytest.org/en/2.0.3/funcargs.html)

* [move to fixtures](https://docs.pytest.org/en/latest/funcarg_compare.html)

# Some Fixture Archeology

## `addfinalizer` -> `yield_fixture` -> `fixture` + `yield`

* teardown in fixtures was historically implemented via a closure passed into `addfinalizer`

* `@pytest.yield_fixture` was the next step up and is [deprecated since 3.0](https://docs.pytest.org/en/latest/yieldfixture.html)

* since then just create a normal fixture and use `yield` instead of `return`

# I WANT MOAR!1!!

* [docs] [my favorite page in the pytest docs](https://docs.pytest.org/en/stable/reference.html) (but the whole docs are really good)

* [tutorial] [simple, rapid and fun testing with pytest](https://www.youtube.com/watch?v=CMuSn9cofbI) by Florian Bruhin 

* [talk] [Introduction to automated testing with pytest](https://www.youtube.com/watch?v=4R0dcsNrrAI) by Raphael Pierzina

* [book] [pytest Quick Start Guide](https://www.packtpub.com/eu/web-development/pytest-quick-start-guide) by Bruno Oliveira (pytest core dev)

* [book] [Python Testing with pytest](https://pragprog.com/titles/bopytest/) by Brian Okken (also check out [his podcast](https://testandcode.com/))

# Thank You For Listening!

![](_img/flopsi-in-a-box.jpg)